# Simulated Annealing

Simulated annealing algorithm:
* Select an initial solution
* Select  the  temperature  change  counter  k=0
* Select  a  temperature  cooling  schedule
* Select an initial temperature
* Select a repetition schedule, that defines the number of iterations executed at eachtemperature
* For t in \[$t_{max}$, $t_{min}$\]:
    * For m in \[0, $m_{max}$\]
        * Generate a solution
        * Calculate energy diff
        * If energy diff < 0 then take new state
        * If energy diff > 0 then take new state with probability exp(-energy diff/t)

## Imports

In [1]:
import json
from math import exp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from random import random, uniform
import seaborn as sns
import sys
import time

%matplotlib inline
sys.path.append('../../../stationsim/')
from stationsim_gcs_model import Model

## Constants

In [2]:
METROPOLIS = 1
TIME_TO_COMPLETION = 5687
MAX_ACTIVE_POPULATION = 85
NEIGHBOURHOOD = 0.5
TEMPERATURE_DROP = 4

## Classes

In [3]:
class State():
    def __init__(self, model_params):
        self.model_params = model_params
        self.model = Model(**self.model_params)
        self.population_over_time = [self.model.pop_active]

    def get_energy(self):
        return abs(self.__get_time_to_completion() - TIME_TO_COMPLETION)
        # return abs(self.__get_max_active_population() - MAX_ACTIVE_POPULATION)

    def __get_max_active_population(self):
        return max(self.population_over_time)

    def __get_time_to_completion(self):
        return self.model.finish_step_id

    def run_model(self):
        for _ in range(self.model.step_limit):
            self.model.step()
            self.population_over_time.append(self.model.pop_active)

## Functions

In [4]:
def get_next_state(current_state):
    # Get current parameter values
    model_params = current_state.model_params
    
    # Do something to perturb the variable of interest
    new_model_params = model_params.copy()
    perturbation = uniform(-NEIGHBOURHOOD, NEIGHBOURHOOD)
    new_model_params['birth_rate'] += perturbation
    
    # Make new state and run model for new values
    state = State(new_model_params)
    state.run_model()
    return state

In [5]:
def simulated_annealing(initial_state, temp_min=0, temp_max=100):
    params = [initial_state.model_params]
    state = initial_state

    for temp in range(temp_max, temp_min, -TEMPERATURE_DROP):
        for i in range(METROPOLIS):
            print(f'Run {i} for temperature {temp}')
            current_energy = state.get_energy()
            next_state = get_next_state(state)
            next_energy = next_state.get_energy()
            energy_change = next_energy - current_energy

            if energy_change < 0:
                print('Change state - greed')
                state = next_state
            elif exp(energy_change / temp) > random():
                print('Change state - exploration')
                state = next_state
            params.append(state.model_params)
    return initial_state, state, params

## Run

In [6]:
scaling_factor = 25/14
speed_mean = 1.6026400144010877 / scaling_factor
speed_std = 0.6642343305178546 / scaling_factor
speed_min = 0.31125359137714953 / scaling_factor

print(f'mean: {speed_mean}, std: {speed_std}, min: {speed_min}')

mean: 0.8974784080646091, std: 0.3719712250899986, min: 0.17430201117120372


In [7]:
model_params = {'station': 'Grand_Central',
                'speed_mean': speed_mean,
                'speed_std:': speed_std,
                'speed_min': speed_min,
                'step_limit': 20000,
                'do_print': False,
                'pop_total': 274,
                'birth_rate': 1.5}

state = State(model_params)
state.run_model()

../../../stationsim/stationsim_gcs_model.py:573: RuntimeWarning: speed_std: is not a model parameter.
  warnings.warn(f'{key} is not a model parameter.',


In [8]:
initial_state, final_state, param_list = simulated_annealing(state)

Run 0 for temperature 100


TypeError: random() takes no arguments (2 given)

## Bibliography